Data Preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
file_path = "/content/drive/MyDrive/small dataset.xlsx"
df = pd.read_excel(file_path)

In [ ]:
numeric_cols = df.select_dtypes(include='number').columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

categorical_cols = df.select_dtypes(exclude='number').columns
df[categorical_cols] = df[categorical_cols].apply(lambda x: x.fillna(x.mode().iloc[0]), axis=0)


In [ ]:
df.head()

,id,comment_text,split,created_date,publication_id,parent_id,article_id,rating,funny,wow,...,white,asian,latino,other_race_or_ethnicity,physical_disability,intellectual_or_learning_disability,psychiatric_or_mental_illness,other_disability,identity_annotator_count,toxicity_annotator_count
0,1083994,He got his money... now he lies in wait till a...,train,2017-03-06 15:21:53.675241+00,21,3.630264e+06,317120,approved,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,67
1,650904,Mad dog will surely put the liberals in mental...,train,2016-12-02 16:44:21.329535+00,21,3.630264e+06,154086,approved,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,76
2,5902188,And Trump continues his lifelong cowardice by ...,train,2017-09-05 19:05:32.341360+00,55,3.630264e+06,374342,approved,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,63
3,7084460,"""while arresting a man for resisting arrest"".\...",test,2016-11-01 16:53:33.561631+00,13,3.630264e+06,149218,approved,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,76
4,5410943,Tucker and Paul are both total bad ass mofo's.,train,2017-06-14 05:08:21.997315+00,21,3.630264e+06,344096,approved,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,80


In [ ]:
cor=['severe_toxicity','obscene','sexual_explicit','identity_attack','insult','threat']#CHANGE
for i in cor:
     correlation_with_toxicity = df['toxicity'].corr(df[i])
     print(f'toxicity and {i}: {correlation_with_toxicity}')

toxicity and severe_toxicity: 0.2944886134659981
toxicity and obscene: 0.36774093788361173
toxicity and sexual_explicit: 0.03205390290118241
toxicity and identity_attack: 0.02287439337895687
toxicity and insult: 0.8788786634523773
toxicity and threat: 0.10221129722700215


In [ ]:
df=df[['severe_toxicity','obscene','sexual_explicit','identity_attack','insult','threat','toxicity','rating','comment_text']]#'rating_approved'	,'rating_rejected']]
df.head()

,severe_toxicity,obscene,sexual_explicit,identity_attack,insult,threat,toxicity,rating,comment_text
0,0.044776,0.089552,0.014925,0.000000,0.343284,0.014925,0.373134,approved,He got his money... now he lies in wait till a...
1,0.013158,0.065789,0.013158,0.092105,0.565789,0.065789,0.605263,approved,Mad dog will surely put the liberals in mental...
2,0.015873,0.031746,0.000000,0.047619,0.666667,0.000000,0.666667,approved,And Trump continues his lifelong cowardice by ...
3,0.065789,0.552632,0.592105,0.000000,0.684211,0.105263,0.815789,approved,"""while arresting a man for resisting arrest"".\..."
4,0.037500,0.337500,0.275000,0.037500,0.487500,0.000000,0.550000,approved,Tucker and Paul are both total bad ass mofo's.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.preprocessing import LabelEncoder

nltk.download('punkt')
nltk.download('stopwords')

def tokenize_and_preprocess(text):
    text = text.lower()
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return tokens

df['tokenized_text'] = df['comment_text'].apply(tokenize_and_preprocess)

print(df[['comment_text', 'tokenized_text']])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


                                          comment_text  \
0    He got his money... now he lies in wait till a...   
1    Mad dog will surely put the liberals in mental...   
2    And Trump continues his lifelong cowardice by ...   
3    "while arresting a man for resisting arrest".\...   
4       Tucker and Paul are both total bad ass mofo's.   
..                                                 ...   
494                      Here, you dropped your brain!   
495  Tourism, not drilling, mining, cattle ranching...   
496  Oh look a leftist that hates cops and brands p...   
497  our trade deficit with the US is 11 billion  o...   
498  Oh Wave, how wrong we were to assume this kill...   

                                        tokenized_text  
0    [got, money, lies, wait, till, election, yrs, ...  
1    [mad, dog, surely, put, liberals, mental, hosp...  
2    [trump, continues, lifelong, cowardice, making...  
3    [arresting, man, resisting, arrest, ca, see, p...  
4                [

Naive Bayes

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

df['rating_approved'] = (df['rating'] == 'approved').astype(int)
X = df[['severe_toxicity', 'obscene', 'sexual_explicit', 'identity_attack', 'insult', 'threat']]
y = df['rating_approved']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = MultinomialNB()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Confusion Matrix:\n', conf_matrix)
print('Classification Report:\n', classification_rep)


Accuracy: 0.77
Confusion Matrix:
 [[ 0 23]
 [ 0 77]]
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        23
           1       0.77      1.00      0.87        77

    accuracy                           0.77       100
   macro avg       0.39      0.50      0.44       100
weighted avg       0.59      0.77      0.67       100



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Perceptron

In [ ]:
df['rating_approved'] = (df['rating'] == 'approved').astype(int)
df['rating_not_approved'] = (df['rating'] != 'approved').astype(int)
X = df[['severe_toxicity', 'obscene', 'sexual_explicit', 'identity_attack', 'insult', 'threat']]
y = df['rating_approved']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

perceptron = Perceptron()
perceptron.fit(X_train, y_train)
y_pred = perceptron.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.52


SVM

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from scipy.sparse import hstack

import pandas as pd

df['rating_approved'] = (df['rating'] == 'approved').astype(int)
numerical_features = df[['severe_toxicity', 'obscene', 'sexual_explicit', 'identity_attack', 'insult', 'threat']]
comments = df['comment_text']

vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
comments_tfidf = vectorizer.fit_transform(comments)
X = hstack([numerical_features, comments_tfidf])

y = df['rating_approved']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

svm_model = SVC(kernel='linear', C=1.0)
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)


Accuracy: 0.75
Classification Report:
               precision    recall  f1-score   support

           0       0.33      0.09      0.14        23
           1       0.78      0.95      0.85        77

    accuracy                           0.75       100
   macro avg       0.55      0.52      0.50       100
weighted avg       0.67      0.75      0.69       100



Logistic Regression

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from scipy.sparse import hstack
import pandas as pd

df['rating_approved'] = (df['rating'] == 'approved').astype(int)
comments = df['comment_text']
numerical_features = df[['severe_toxicity', 'obscene', 'sexual_explicit', 'identity_attack', 'insult', 'threat']]

vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
comments_tfidf = vectorizer.fit_transform(comments)
X = hstack([numerical_features.values, comments_tfidf])
y = df['rating_approved']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

logreg_model = LogisticRegression()
logreg_model.fit(X_train, y_train)
y_pred = logreg_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)



Accuracy: 0.77
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        23
           1       0.77      1.00      0.87        77

    accuracy                           0.77       100
   macro avg       0.39      0.50      0.44       100
weighted avg       0.59      0.77      0.67       100



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


KNN

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
import pandas as pd
from scipy.sparse import hstack

df['rating_approved'] = (df['rating'] == 'approved').astype(int)
comments = df['comment_text']
numerical_features = df[['severe_toxicity', 'obscene', 'sexual_explicit', 'identity_attack', 'insult', 'threat']]

vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
comments_tfidf = vectorizer.fit_transform(comments)
X = hstack([numerical_features.values, comments_tfidf])

scaler = StandardScaler(with_mean=False)
X = scaler.fit_transform(X)
y = df['rating_approved']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)
y_pred = knn_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)


Accuracy: 0.72
Classification Report:
               precision    recall  f1-score   support

           0       0.31      0.17      0.22        23
           1       0.78      0.88      0.83        77

    accuracy                           0.72       100
   macro avg       0.54      0.53      0.53       100
weighted avg       0.67      0.72      0.69       100



Random Forest Classifier

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import pandas as pd

file_path = "/content/drive/MyDrive/small dataset.xlsx"
df = pd.read_excel(file_path)

X = df['comment_text']
y = df['toxicity']

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_tfidf, y_train)
y_pred = rf_model.predict(X_test_tfidf)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

user_comment = input("Enter a comment: ")
user_comment_tfidf = vectorizer.transform([user_comment])
user_prediction = rf_model.predict(user_comment_tfidf)
if user_prediction[0] == 1:
    print("Toxic Comment")
else:
    print("Non-Toxic Comment")


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

Accuracy: 0.01
Classification Report:
               precision    recall  f1-score   support

           8       0.00      0.00      0.00         1
           9       0.00      0.00      0.00         1
          14       0.00      0.00      0.00         3
          17       0.00      0.00      0.00         7
          20       0.00      0.00      0.00         1
          22       0.00      0.00      0.00         0
          23       0.00      0.00      0.00         1
          27       0.00      0.00      0.00         0
          28       0.00      0.00      0.00         1
          35       0.00      0.00      0.00         1
          36       0.00      0.00      0.00         0
          37       0.00      0.00      0.00         1
          38       0.00      0.00      0.00         1
          39       0.00      0.00      0.00         0
          41       0.00      0.00      0.00         1
          44       0.00      0.00      0.00         1
          46       0.00      0.00      0.0

Ask the user to input a comment, the predict whether it is toxic or not

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from scipy.sparse import hstack
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

file_path = "/content/drive/MyDrive/small dataset.xlsx"
df = pd.read_excel(file_path)

df['rating_approved'] = (df['rating'] == 'approved').astype(int)
comments = df['comment_text']

vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
comments_tfidf = vectorizer.fit_transform(comments)
X = hstack([comments_tfidf])
y = df['rating_approved']

logreg_model = LogisticRegression()
logreg_model.fit(X, y)
logreg_model = LogisticRegression(C=1.0)
logreg_model.fit(X, y)

user_comment = input("Enter a comment: ")
user_comment_tfidf = vectorizer.transform([user_comment])
prediction = logreg_model.predict(user_comment_tfidf)
if prediction[0] == 1:
    print("Toxic Comment")
else:
    print("Non-Toxic Comment")
